In [ ]:
#=========preparation==========#
import torch as torch
import torch.nn as nn
import torch.optim as optim
import torchvision as torchvision
from torchvision import transforms as transforms
import sys

In [ ]:
# =========CIFAR-10 with data augmentation==========#
def dataset_cifar10(batch_size = 200):
    transform_train = transforms.Compose([  # 数据集增强 使用cifar100时 停用数据集增强
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
    ])
    transform_test = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

    if sys.platform.startswith("win"):
        # 0表示不用额外的进程来加速读取数据
        num_workers = 2
    else:
        num_workers = 4
    # 加载数据集
    trainset = torchvision.datasets.CIFAR10(root="data", train=True, download=True, transform=transform_train)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=num_workers)
    testset = torchvision.datasets.CIFAR10(root="data", train=False, download=True, transform=transform_test)
    testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=num_workers)
    return trainloader, testloader, batch_size